# Library

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load data, train/test split, x/y split

In [7]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Data shape and preprocess

In [28]:
# data shape
shape = x_train.shape[1:]

# preprocess (reshape x)
X_train = x_train.reshape(-1, np.prod(shape))
X_test = x_test.reshape(-1, np.prod(shape))

# preprocess (normalize x)
X_train = X_train / 255.
X_test = X_test / 255.

# preprocess (categorical y)
print('y_train shape:', y_train.shape) # (50000, 1)
print('y_test shape:', y_test.shape) # (10000, 1)

    # tf.one_hot 이용
print("tf.one_hot 이용")
tf.one_hot?
tf_Y_train = tf.one_hot(y_train[:,0], 10)
with tf.Session() as sess:
    print(tf_Y_train.shape, tf_Y_train[:3])
    print(y_train[:3])
    print(sess.run(tf_Y_train[:3]))
    
    # numpy 이용
print("numpy 이용")
Y_train = np.eye(10)[y_train.reshape(-1)]
print(Y_train.shape)
print(Y_train[:3])

y_train shape: (50000, 1)
y_test shape: (10000, 1)
tf.one_hot 이용
(50000, 10) Tensor("strided_slice_16:0", shape=(3, 10), dtype=float32)
[[6]
 [9]
 [9]]
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
numpy 이용
(50000, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# Hyper parameter setting

In [7]:
# hyper parameter setting

# Model

In [46]:
# model

# loss function 짜는 법
# 1. y placeholder 와 pred 값의 shape 생각해보기 -> (?, 10)
# 2. sum_k( y_k * log(pred_k) ) 의 의미 생각해보기
# 3. 답지에서의 구현방법: y * pred = [ [0, 0, log(pred_k), 0, 0, 0,..], ... ]
#                         shape (?, 10)
#                       : tf.reduce_sum(_, axis=1) = [ log(pred_k), ... ]
#                         shape (?, )
#                       : tf.reduce_mean(_, axis=0) = E[log(pred_k)]
#                         shape ()

# placeholder
tf_X = tf.placeholder(tf.float32, [None, np.prod(shape)])
tf_Y = tf.placeholder(tf.float32, [None, 10])
# use library
y_logit = tf.layers.dense(tf_X, 10)
y_pred = tf.nn.softmax(y_logit)

loss = tf.losses.softmax_cross_entropy(tf_Y, y_logit)
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# Train

In [53]:
# training
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(10):
        print(i)
        sess.run(optimizer, feed_dict={tf_X:X_train, tf_Y:Y_train})
    y_pred_numpy_onehot = sess.run(y_pred, feed_dict={tf_X:X_test})

0
1
2
3
4
5
6
7
8
9


# Confusion matrix

In [56]:
# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred_numpy_label = np.argmax(y_pred_numpy_onehot, axis=1)
confusion_matrix(y_test, y_pred_numpy_label)

array([[ 17,   0,   2, 152,  15,   0,  27, 565,  11, 211],
       [ 46,   3,  13, 248,  27,   0, 109, 318,   7, 229],
       [ 41,   1,   3, 127,  49,   0,  47, 549,   6, 177],
       [ 42,   6,   4, 179,  40,   0,  41, 466,  23, 199],
       [ 27,   0,   1, 158,  19,   0,  32, 570,   6, 187],
       [ 41,   1,   3, 165,  24,   0,  38, 457,  35, 236],
       [ 32,   2,   3, 210,  54,   0,  72, 415,  14, 198],
       [ 41,   0,   0, 133,  32,   0,  57, 421,  17, 299],
       [ 25,   3,   0, 295,  21,   0,  47, 284,  14, 311],
       [ 56,   1,   0, 256,  34,   0,  65, 201,   8, 379]])